In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Data Imputation/Output')
cur_date = "020923"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
housing_df = data.frame(read_excel("Input/Housing_Data_020923.xlsx", sheet = 2))

# making some of the cols numeric
housing_df[,4:12] = data.frame(apply(housing_df[,4:12], 2,as.numeric))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(housing_df[, 4:12], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(housing_df[, 4:12], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(housing_df[, 4:12], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(housing_df[, 4:12], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(housing_df[, 4:12], 2, as.numeric):
“NAs introduced by coercion”


In [2]:
head(housing_df)

,GEO_ID,Census_Tract,County,Total_Housing_Units,Housing_Density,Total_Occupied_Housing_Units,Total_Owner_Occupied_Housing_Units,Median_Year_Structure_Built,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,2006,31.54848,1757,851,1949,824,149600,614,27.9
2,14000US37001020200,202.00,Alamance,1943,26.03618,1675,881,1961,881,83400,609,31.5
3,14000US37001020300,203.00,Alamance,3254,71.60011,2930,2015,1971,1980,91500,763,37.7
4,14000US37001020400,204.00,Alamance,2590,53.47674,2327,1510,1974,1487,96900,659,34.3
5,14000US37001020501,205.01,Alamance,1435,122.03583,1280,990,1973,932,137400,584,46.7
6,14000US37001020502,205.02,Alamance,1484,471.34609,1275,666,1980,666,73500,672,35.9


# Variable & GEO Id Filtering

Removing variables and GEO ids that have less than 25% of data prior to imputation

In [3]:
# only keeping variables with at least 25% of data 
variable_presence_df = housing_df %>% 
    pivot_longer(cols = 5:8, names_to = "Variable", values_to = "Value") %>%
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Variable) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

Variable,Variable_Presence_Percent
<chr>,<dbl>
Median_Year_Structure_Built,98.43534
Housing_Density,99.86194
Total_Occupied_Housing_Units,100.00000
Total_Owner_Occupied_Housing_Units,100.00000


In [4]:
# geo id percentages
geo_id_percent_occurence = apply(housing_df[5:8], 1, function(x) (length(na.omit(x))/length(x)) * 100)
                                 
# creating a df and adding back in geo ids
id_presence_df = data.frame(matrix(ncol = 2, nrow = length(geo_id_percent_occurence)))
colnames(id_presence_df) = c("GEO_ID", "GEO_ID_Presence_Percentage")
id_presence_df$GEO_ID = housing_df$GEO_ID
id_presence_df$GEO_ID_Presence_Percentage = geo_id_percent_occurence
                                 
id_presence_df = id_presence_df %>%
    filter(GEO_ID_Presence_Percentage < 25) %>%
    arrange(GEO_ID_Presence_Percentage)
                                 
head(id_presence_df)

GEO_ID,GEO_ID_Presence_Percentage
<chr>,<dbl>


In [5]:
# number of geo ids to be removed
dim(id_presence_df)[1]

[1] 0

In [6]:
# getting 0 geo ids to be removed
removed_geo_ids = id_presence_df$GEO_ID

`%notin%` <- Negate(`%in%`)
filtered_housing_df = housing_df %>%
    filter(GEO_ID %notin% removed_geo_ids)

head(filtered_housing_df)
dim(filtered_housing_df)

,GEO_ID,Census_Tract,County,Total_Housing_Units,Housing_Density,Total_Occupied_Housing_Units,Total_Owner_Occupied_Housing_Units,Median_Year_Structure_Built,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,2006,31.54848,1757,851,1949,824,149600,614,27.9
2,14000US37001020200,202.00,Alamance,1943,26.03618,1675,881,1961,881,83400,609,31.5
3,14000US37001020300,203.00,Alamance,3254,71.60011,2930,2015,1971,1980,91500,763,37.7
4,14000US37001020400,204.00,Alamance,2590,53.47674,2327,1510,1974,1487,96900,659,34.3
5,14000US37001020501,205.01,Alamance,1435,122.03583,1280,990,1973,932,137400,584,46.7
6,14000US37001020502,205.02,Alamance,1484,471.34609,1275,666,1980,666,73500,672,35.9


[1] 2173   12

All variables and geo ids will be kept, leaving us with 2,173 geo ids and 12 variables. 

# Random Forest Imputation
Generating missing values using random forest (RF).

In [7]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df){

    imputed_RF_object = missForest(as.matrix(preimputed_df[,4:12]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_values_df = data.frame(cbind(preimputed_df[,1:3], imputed_RF_df)) 

    
    return(imputed_values_df)
}

# calling function
imputed_df = random_forest_imputation(filtered_housing_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Total_Housing_Units,Housing_Density,Total_Occupied_Housing_Units,Total_Owner_Occupied_Housing_Units,Median_Year_Structure_Built,Owner_Occupied_Housing_Units_Less_than_HPI,Median_House_Value,Median_Gross_Rent,Median_Gross_Rent_Percentage_of_Household_Income
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,2006,31.54848,1757,851,1949,824,149600,614,27.9
2,14000US37001020200,202.00,Alamance,1943,26.03618,1675,881,1961,881,83400,609,31.5
3,14000US37001020300,203.00,Alamance,3254,71.60011,2930,2015,1971,1980,91500,763,37.7
4,14000US37001020400,204.00,Alamance,2590,53.47674,2327,1510,1974,1487,96900,659,34.3
5,14000US37001020501,205.01,Alamance,1435,122.03583,1280,990,1973,932,137400,584,46.7
6,14000US37001020502,205.02,Alamance,1484,471.34609,1275,666,1980,666,73500,672,35.9


In [8]:
# double checking dimensions of the final df
dim(imputed_df)

[1] 2173   12

In [9]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_Housing_Data.csv"), row.names = FALSE)